# Trade network analysis
**Brian Dew (brianwdew@gmail.com)**

**04_imfdata.ipynb**

Using the IMF API, data on exports, prices, and exchange rates are collected for all available countries during 2008-2014.

#### Import packages

In [1]:
import requests                                             # For requesting json data from the url
import pandas as pd                                         # pandas dataframes used for convenience
import os                                                   # change current directory in next line
os.chdir('C:/Working/trade_network/data/')

#### settings for API request

In [2]:
webserv = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/' # the main URL for the JSON rest API
method = 'CompactData/'                                     # Data is stored here. Datastructure has info
series = 'IFS'                                              # International Financial Statistics series
freq = 'A'                                                  # Annual
inds = {'x': 'TXG_FOB_USD', 'q': 'EREER_IX', 'p': 'TMG_D_USD_CIF_IX'}  # Set of IFS indicators of interest
date = '?startPeriod=2008&endPeriod=2014'                   # Date range of interest

#### Loop with API request for each indicators

In [18]:
fd = {} # dictionary for saving each series from inds above
for k, v in inds.iteritems(): # k is the key and v is the value
    url = webserv+method+series+'/'+freq+'..'+v+'.'+date # print url to see
    df = pd.DataFrame(requests.get(url).json()['CompactData']['DataSet']['Series']).set_index('@REF_AREA')
    df = df[df['Obs'].apply(lambda x: isinstance(x, list))] # drops empties
    d = {} # temporary dict to save country by country dataframes
    for c in df.index.values: # index values are countries (@REF_AREA) as set above
        d[c] = pd.DataFrame(df.loc[c]['Obs']).rename(columns={'@TIME_PERIOD':'date'})
        d[c]['date'] = pd.to_datetime(d[c]['date'].values) # set dates as datetime for index
    fd[k]= pd.concat(d, axis=0).reset_index().set_index(['level_0','date']).drop('level_1', 1)
merged = pd.concat(fd, axis=1) # combine all series to one merged dataframe

http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/A..EREER_IX.?startPeriod=2008&endPeriod=2014
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/A..TXG_FOB_USD.?startPeriod=2008&endPeriod=2014
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/A..TMG_D_USD_CIF_IX.?startPeriod=2008&endPeriod=2014


p                     q            x
                   @BASE_YEAR @OBS_VALUE @OBS_VALUE   @OBS_VALUE
level_0 date                                                    
ZW      2010-01-01        NaN        NaN        NaN     3199.231
        2011-01-01        NaN        NaN        NaN     3512.125
        2012-01-01        NaN        NaN        NaN         3800
        2013-01-01        NaN        NaN        NaN  3551.886684
        2014-01-01        NaN        NaN        NaN         3438